## Deliverable 3

In [1]:
!pip install keras-tuner

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow import keras

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_values = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in replace_values:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_value_counts = application_df['CLASSIFICATION'].value_counts()
class_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2500        1
C2600        1
C2190        1
C1245        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_value_counts[class_value_counts < 500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
app_categories = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
app_categories

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_categories]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_categories)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = app_categories)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# # Create a method that creates a new Sequential model with hyperparameter options
# #def create_model(hp):
#     nn_model = tf.keras.models.Sequential()

#     # Allow kerastuner to decide which activation function to use in hidden layers
#     activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
#     # Allow kerastuner to decide number of neurons in first layer
#     nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#         min_value=1,
#         max_value=10,
#         step=2), activation=activation, input_dim=44))

#     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#     for i in range(hp.Int('num_layers', 1, 6)):
#         nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
#             min_value=1,
#             max_value=10,
#             step=2),
#             activation=activation))
    
#     nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#     # Compile the model
#     nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
#     return nn_model

In [27]:
# Import the kerastuner library
#import kerastuner as kt

#tuner = kt.Hyperband(
#    create_model,
#    objective="val_accuracy",
#    max_epochs=20,
#    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


In [28]:
# Run the kerastuner search for best hyperparameters
#tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Epoch 1/3
804/804 [==============================] - ETA: 0s - loss: 0.6979 - accuracy: 0.50 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s - loss: 0.6916 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318 - val_loss: 0.6908 - val_accuracy: 0.5343
Epoch 2/3
804/804 [==============================] - ETA: 0s - loss: 0.6748 - accuracy: 0.65 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6921 - accuracy: 0.52 - ETA: 0s - loss: 0.6920 - accuracy: 0.52 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s 

Epoch 1/3
804/804 [==============================] - ETA: 0s - loss: 0.8086 - accuracy: 0.37 - ETA: 0s - loss: 0.7409 - accuracy: 0.47 - ETA: 0s - loss: 0.7359 - accuracy: 0.47 - ETA: 0s - loss: 0.7291 - accuracy: 0.47 - ETA: 0s - loss: 0.7246 - accuracy: 0.47 - ETA: 0s - loss: 0.7205 - accuracy: 0.47 - ETA: 0s - loss: 0.7169 - accuracy: 0.47 - ETA: 0s - loss: 0.7145 - accuracy: 0.46 - ETA: 0s - loss: 0.7120 - accuracy: 0.46 - ETA: 0s - loss: 0.7100 - accuracy: 0.46 - ETA: 0s - loss: 0.7082 - accuracy: 0.47 - ETA: 0s - loss: 0.7066 - accuracy: 0.48 - ETA: 0s - loss: 0.7053 - accuracy: 0.48 - 1s 1ms/step - loss: 0.7048 - accuracy: 0.4870 - val_loss: 0.6904 - val_accuracy: 0.5343
Epoch 2/3
804/804 [==============================] - ETA: 0s - loss: 0.6913 - accuracy: 0.50 - ETA: 0s - loss: 0.6913 - accuracy: 0.51 - ETA: 0s - loss: 0.6905 - accuracy: 0.52 - ETA: 0s - loss: 0.6897 - accuracy: 0.53 - ETA: 0s - loss: 0.6891 - accuracy: 0.53 - ETA: 0s - loss: 0.6885 - accuracy: 0.53 - ETA: 0s 

Epoch 1/3
804/804 [==============================] - ETA: 0s - loss: 0.7036 - accuracy: 0.40 - ETA: 0s - loss: 0.6851 - accuracy: 0.54 - ETA: 0s - loss: 0.6842 - accuracy: 0.53 - ETA: 0s - loss: 0.6806 - accuracy: 0.55 - ETA: 0s - loss: 0.6753 - accuracy: 0.57 - ETA: 0s - loss: 0.6726 - accuracy: 0.58 - ETA: 0s - loss: 0.6669 - accuracy: 0.60 - ETA: 0s - loss: 0.6625 - accuracy: 0.61 - ETA: 0s - loss: 0.6581 - accuracy: 0.62 - ETA: 0s - loss: 0.6537 - accuracy: 0.62 - ETA: 0s - loss: 0.6506 - accuracy: 0.63 - ETA: 0s - loss: 0.6474 - accuracy: 0.64 - ETA: 0s - loss: 0.6438 - accuracy: 0.64 - ETA: 0s - loss: 0.6418 - accuracy: 0.65 - ETA: 0s - loss: 0.6390 - accuracy: 0.65 - 1s 1ms/step - loss: 0.6366 - accuracy: 0.6576 - val_loss: 0.5905 - val_accuracy: 0.7125
Epoch 2/3
804/804 [==============================] - ETA: 0s - loss: 0.5171 - accuracy: 0.84 - ETA: 0s - loss: 0.5854 - accuracy: 0.73 - ETA: 0s - loss: 0.5958 - accuracy: 0.71 - ETA: 0s - loss: 0.5914 - accuracy: 0.72 - ETA: 0s 

Epoch 1/3
804/804 [==============================] - ETA: 0s - loss: 0.6669 - accuracy: 0.59 - ETA: 0s - loss: 0.6690 - accuracy: 0.59 - ETA: 0s - loss: 0.6527 - accuracy: 0.63 - ETA: 0s - loss: 0.6416 - accuracy: 0.65 - ETA: 0s - loss: 0.6318 - accuracy: 0.66 - ETA: 0s - loss: 0.6241 - accuracy: 0.67 - ETA: 0s - loss: 0.6212 - accuracy: 0.67 - ETA: 0s - loss: 0.6166 - accuracy: 0.68 - ETA: 0s - loss: 0.6138 - accuracy: 0.68 - ETA: 0s - loss: 0.6110 - accuracy: 0.68 - ETA: 0s - loss: 0.6083 - accuracy: 0.69 - ETA: 0s - loss: 0.6058 - accuracy: 0.69 - ETA: 0s - loss: 0.6029 - accuracy: 0.69 - ETA: 0s - loss: 0.6015 - accuracy: 0.69 - 1s 1ms/step - loss: 0.5988 - accuracy: 0.7003 - val_loss: 0.5757 - val_accuracy: 0.7183
Epoch 2/3
804/804 [==============================] - ETA: 0s - loss: 0.5444 - accuracy: 0.75 - ETA: 0s - loss: 0.5784 - accuracy: 0.71 - ETA: 0s - loss: 0.5708 - accuracy: 0.71 - ETA: 0s - loss: 0.5668 - accuracy: 0.72 - ETA: 0s - loss: 0.5697 - accuracy: 0.72 - ETA: 0s 

Epoch 1/3
804/804 [==============================] - ETA: 0s - loss: 0.6938 - accuracy: 0.50 - ETA: 0s - loss: 0.6928 - accuracy: 0.52 - ETA: 0s - loss: 0.6928 - accuracy: 0.51 - ETA: 0s - loss: 0.6921 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s - loss: 0.6900 - accuracy: 0.53 - ETA: 0s - loss: 0.6881 - accuracy: 0.53 - ETA: 0s - loss: 0.6851 - accuracy: 0.54 - ETA: 0s - loss: 0.6810 - accuracy: 0.56 - ETA: 0s - loss: 0.6768 - accuracy: 0.57 - 1s 1ms/step - loss: 0.6753 - accuracy: 0.5811 - val_loss: 0.6179 - val_accuracy: 0.7250
Epoch 2/3
804/804 [==============================] - ETA: 0s - loss: 0.5721 - accuracy: 0.84 - ETA: 0s - loss: 0.6039 - accuracy: 0.74 - ETA: 0s - loss: 0.6014 - accuracy: 0.73 - ETA: 0s - loss: 0.6031 - accuracy: 0.72 - ETA: 0s - loss: 0.5985 - accuracy: 0.73 - ETA: 0s - loss: 0.5959 - accuracy: 0.72 - ETA: 0s - loss: 0.5912 - accuracy: 0.73 - ETA: 0s - loss: 0.5899 - accuracy: 0.72 - ETA: 0s 

...

Epoch 1/20
804/804 [==============================] - 1s 1ms/step - loss: 0.6327 - accuracy: 0.6649 - val_loss: 0.5841 - val_accuracy: 0.7224
Epoch 2/20
804/804 [==============================] - ETA: 0s - loss: 0.5726 - accuracy: 0.71 - ETA: 0s - loss: 0.5813 - accuracy: 0.73 - ETA: 0s - loss: 0.5822 - accuracy: 0.72 - ETA: 0s - loss: 0.5777 - accuracy: 0.73 - ETA: 0s - loss: 0.5798 - accuracy: 0.72 - ETA: 0s - loss: 0.5786 - accuracy: 0.72 - ETA: 0s - loss: 0.5760 - accuracy: 0.72 - ETA: 0s - loss: 0.5754 - accuracy: 0.72 - ETA: 0s - loss: 0.5745 - accuracy: 0.72 - ETA: 0s - loss: 0.5778 - accuracy: 0.72 - ETA: 0s - loss: 0.5772 - accuracy: 0.72 - ETA: 0s - loss: 0.5765 - accuracy: 0.72 - ETA: 0s - loss: 0.5771 - accuracy: 0.72 - 1s 926us/step - loss: 0.5768 - accuracy: 0.7243 - val_loss: 0.5755 - val_accuracy: 0.7200
Epoch 3/20
804/804 [==============================] - ETA: 0s - loss: 0.4632 - accuracy: 0.81 - ETA: 0s - loss: 0.5681 - accuracy: 0.72 - ETA: 0s - loss: 0.5740 - accur

804/804 [==============================] - ETA: 0s - loss: 0.5578 - accuracy: 0.68 - ETA: 0s - loss: 0.5434 - accuracy: 0.74 - ETA: 0s - loss: 0.5431 - accuracy: 0.74 - ETA: 0s - loss: 0.5423 - accuracy: 0.74 - ETA: 0s - loss: 0.5455 - accuracy: 0.73 - ETA: 0s - loss: 0.5493 - accuracy: 0.73 - ETA: 0s - loss: 0.5527 - accuracy: 0.73 - ETA: 0s - loss: 0.5542 - accuracy: 0.73 - ETA: 0s - loss: 0.5514 - accuracy: 0.73 - ETA: 0s - loss: 0.5500 - accuracy: 0.73 - ETA: 0s - loss: 0.5533 - accuracy: 0.73 - ETA: 0s - loss: 0.5540 - accuracy: 0.73 - ETA: 0s - loss: 0.5553 - accuracy: 0.73 - 1s 988us/step - loss: 0.5559 - accuracy: 0.7303 - val_loss: 0.5604 - val_accuracy: 0.7261
Epoch 13/20
804/804 [==============================] - ETA: 0s - loss: 0.4709 - accuracy: 0.78 - ETA: 0s - loss: 0.5540 - accuracy: 0.73 - ETA: 0s - loss: 0.5585 - accuracy: 0.72 - ETA: 0s - loss: 0.5573 - accuracy: 0.72 - ETA: 0s - loss: 0.5581 - accuracy: 0.72 - ETA: 0s - loss: 0.5596 - accuracy: 0.72 - ETA: 0s - loss

Epoch 1/20
804/804 [==============================] - ETA: 0s - loss: 0.6919 - accuracy: 0.56 - ETA: 0s - loss: 0.6938 - accuracy: 0.49 - ETA: 0s - loss: 0.6932 - accuracy: 0.51 - ETA: 0s - loss: 0.6929 - accuracy: 0.51 - ETA: 0s - loss: 0.6924 - accuracy: 0.51 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6883 - accuracy: 0.55 - ETA: 0s - loss: 0.6821 - accuracy: 0.56 - ETA: 0s - loss: 0.6736 - accuracy: 0.58 - ETA: 0s - loss: 0.6677 - accuracy: 0.60 - ETA: 0s - loss: 0.6627 - accuracy: 0.61 - ETA: 0s - loss: 0.6569 - accuracy: 0.62 - 1s 1ms/step - loss: 0.6514 - accuracy: 0.6321 - val_loss: 0.5959 - val_accuracy: 0.7210
Epoch 2/20
804/804 [==============================] - ETA: 0s - loss: 0.5267 - accuracy: 0.75 - ETA: 1s - loss: 0.6078 - accuracy: 0.71 - ETA: 0s - loss: 0.5908 - accuracy: 0.72 - ETA: 0s - loss: 0.5910 - accuracy: 0.71 - ETA: 0s - loss: 0.5915 - accuracy: 0.71 - ETA: 0s - loss: 0.5871 - accuracy: 0.71 - ETA: 0s - loss: 0.5874 - accuracy: 0.71 - ETA: 0

804/804 [==============================] - ETA: 0s - loss: 0.6068 - accuracy: 0.71 - ETA: 0s - loss: 0.5819 - accuracy: 0.71 - ETA: 0s - loss: 0.5611 - accuracy: 0.73 - ETA: 0s - loss: 0.5614 - accuracy: 0.73 - ETA: 0s - loss: 0.5613 - accuracy: 0.73 - ETA: 0s - loss: 0.5625 - accuracy: 0.73 - ETA: 0s - loss: 0.5630 - accuracy: 0.73 - ETA: 0s - loss: 0.5633 - accuracy: 0.72 - ETA: 0s - loss: 0.5629 - accuracy: 0.72 - ETA: 0s - loss: 0.5649 - accuracy: 0.72 - ETA: 0s - loss: 0.5644 - accuracy: 0.72 - ETA: 0s - loss: 0.5645 - accuracy: 0.72 - 1s 854us/step - loss: 0.5640 - accuracy: 0.7282 - val_loss: 0.5672 - val_accuracy: 0.7257
Epoch 14/20
804/804 [==============================] - ETA: 0s - loss: 0.7629 - accuracy: 0.62 - ETA: 0s - loss: 0.5517 - accuracy: 0.73 - ETA: 0s - loss: 0.5611 - accuracy: 0.72 - ETA: 0s - loss: 0.5637 - accuracy: 0.72 - ETA: 0s - loss: 0.5610 - accuracy: 0.72 - ETA: 0s - loss: 0.5608 - accuracy: 0.72 - ETA: 0s - loss: 0.5622 - accuracy: 0.72 - ETA: 0s - loss

INFO:tensorflow:Oracle triggered exit


In [49]:
# Get best model hyperparameters
#best_hyper = tuner.get_best_hyperparameters(1)[0]
#best_hyper.values

{'activation': 'tanh',
 'first_units': 7,
 'num_layers': 4,
 'units_0': 7,
 'units_1': 7,
 'units_2': 9,
 'units_3': 9,
 'units_4': 9,
 'units_5': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '2e4d1bba3af367347a1d3dd3879daa08'}

In [30]:
# Evaluate best model against full test data
#best_model = tuner.get_best_models(1)[0]
#model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
#print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7328
Loss: 0.5540841221809387, Accuracy: 0.7328279614448547


## Attempt 1 - Additional neurons added to hidden layers

In [78]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 100)               4500      
_________________________________________________________________
dense_43 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 51        
Total params: 9,601
Trainable params: 9,601
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [80]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [81]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [82]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - ETA: 0s - loss: 59848.6172 - accuracy: 0.53 - ETA: 0s - loss: 19921.3066 - accuracy: 0.48 - ETA: 0s - loss: 103440.2188 - accuracy: 0.515 - ETA: 0s - loss: 69509.2500 - accuracy: 0.498 - ETA: 0s - loss: 62289.4766 - accuracy: 0.49 - ETA: 0s - loss: 53104.1172 - accuracy: 0.49 - ETA: 0s - loss: 44342.1211 - accuracy: 0.49 - ETA: 0s - loss: 39423.7227 - accuracy: 0.49 - ETA: 0s - loss: 47566.7617 - accuracy: 0.49 - ETA: 0s - loss: 44425.4570 - accuracy: 0.49 - ETA: 0s - loss: 41143.7617 - accuracy: 0.49 - ETA: 0s - loss: 38126.3672 - accuracy: 0.49 - ETA: 0s - loss: 57101.6562 - accuracy: 0.49 - ETA: 0s - loss: 54508.6680 - accuracy: 0.49 - ETA: 0s - loss: 54827.8945 - accuracy: 0.48 - ETA: 0s - loss: 55287.5625 - accuracy: 0.48 - 1s 993us/step - loss: 53373.1094 - accuracy: 0.4860
Epoch 2/100
804/804 [==============================] - ETA: 0s - loss: 487.9547 - accuracy: 0.56 - ETA: 0s - loss: 1278.3209 - accuracy: 0.459 - ETA: 0s -

804/804 [==============================] - ETA: 0s - loss: 16.6871 - accuracy: 0.500 - ETA: 0s - loss: 1348.8248 - accuracy: 0.471 - ETA: 0s - loss: 1908.5079 - accuracy: 0.486 - ETA: 0s - loss: 1264.2114 - accuracy: 0.501 - ETA: 0s - loss: 2452.9058 - accuracy: 0.512 - ETA: 0s - loss: 3359.9807 - accuracy: 0.520 - ETA: 0s - loss: 21061.7129 - accuracy: 0.52 - ETA: 0s - loss: 18787.7969 - accuracy: 0.51 - ETA: 0s - loss: 16727.4609 - accuracy: 0.51 - ETA: 0s - loss: 14905.7803 - accuracy: 0.51 - ETA: 0s - loss: 13314.9014 - accuracy: 0.51 - ETA: 0s - loss: 12146.4375 - accuracy: 0.51 - ETA: 0s - loss: 11198.8857 - accuracy: 0.51 - ETA: 0s - loss: 10514.3760 - accuracy: 0.51 - ETA: 0s - loss: 9954.2568 - accuracy: 0.5197 - ETA: 0s - loss: 9407.2441 - accuracy: 0.519 - ETA: 0s - loss: 8822.6377 - accuracy: 0.520 - ETA: 0s - loss: 8346.7959 - accuracy: 0.520 - ETA: 0s - loss: 7989.3789 - accuracy: 0.518 - 1s 1ms/step - loss: 7839.2300 - accuracy: 0.5171
Epoch 11/100
804/804 [=============

804/804 [==============================] - ETA: 0s - loss: 0.7577 - accuracy: 0.46 - ETA: 0s - loss: 0.6414 - accuracy: 0.63 - ETA: 0s - loss: 0.6356 - accuracy: 0.64 - ETA: 0s - loss: 0.6375 - accuracy: 0.64 - ETA: 0s - loss: 0.6372 - accuracy: 0.64 - ETA: 0s - loss: 0.6355 - accuracy: 0.64 - ETA: 0s - loss: 0.6326 - accuracy: 0.64 - ETA: 0s - loss: 0.6325 - accuracy: 0.64 - ETA: 0s - loss: 0.6322 - accuracy: 0.64 - ETA: 0s - loss: 0.6330 - accuracy: 0.64 - ETA: 0s - loss: 0.6329 - accuracy: 0.64 - ETA: 0s - loss: 0.6317 - accuracy: 0.64 - ETA: 0s - loss: 0.6318 - accuracy: 0.64 - ETA: 0s - loss: 0.6313 - accuracy: 0.64 - 1s 878us/step - loss: 0.6308 - accuracy: 0.6495
Epoch 21/100
804/804 [==============================] - ETA: 0s - loss: 0.5819 - accuracy: 0.65 - ETA: 0s - loss: 0.6326 - accuracy: 0.63 - ETA: 0s - loss: 0.6311 - accuracy: 0.64 - ETA: 0s - loss: 0.6323 - accuracy: 0.64 - ETA: 0s - loss: 0.6346 - accuracy: 0.64 - ETA: 0s - loss: 0.6328 - accuracy: 0.64 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 0.6884 - accuracy: 0.56 - ETA: 0s - loss: 0.6901 - accuracy: 0.54 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6922 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 31/100
804/804 [==============================] - ETA: 0s - loss: 0.7052 - accuracy: 0.43 - ETA: 0s - loss: 0.6906 - accuracy: 0.53 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s - loss:

804/804 [==============================] - ETA: 0s - loss: 0.6772 - accuracy: 0.65 - ETA: 0s - loss: 0.6931 - accuracy: 0.51 - ETA: 0s - loss: 0.6922 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 40/100
804/804 [==============================] - ETA: 0s - loss: 0.6815 - accuracy: 0.59 - ETA: 0s - loss: 0.6910 - accuracy: 0.53 - ETA: 0s - loss:

...

804/804 [==============================] - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6935 - accuracy: 0.50 - ETA: 0s - loss: 0.6930 - accuracy: 0.51 - ETA: 0s - loss: 0.6925 - accuracy: 0.52 - ETA: 0s - loss: 0.6923 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6920 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6920 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 58/100
804/804 [==============================] - ETA: 0s - loss: 0.6710 - accuracy: 0.68 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss:

804/804 [==============================] - ETA: 0s - loss: 0.6699 - accuracy: 0.65 - ETA: 0s - loss: 0.6901 - accuracy: 0.54 - ETA: 0s - loss: 0.6903 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6914 - accuracy: 0.52 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - 1s 997us/step - loss: 0.6913 - accuracy: 0.5318
Epoch 68/100
804/804 [==============================] - ETA: 0s - loss: 0.6950 - accuracy: 0.50 - ETA: 0s - loss: 0.6885 - accuracy: 0.55 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 0.6546 - accuracy: 0.71 - ETA: 0s - loss: 0.6931 - accuracy: 0.52 - ETA: 0s - loss: 0.6929 - accuracy: 0.51 - ETA: 0s - loss: 0.6921 - accuracy: 0.52 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6916 - accuracy: 0.53 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - 1s 956us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 78/100
804/804 [==============================] - ETA: 0s - loss: 0.6950 - accuracy: 0.50 - ETA: 0s - loss: 0.6921 - accuracy: 0.52 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6926 - accuracy: 0.51 - ETA: 0s - los

Epoch 88/100
804/804 [==============================] - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6905 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - 1s 957us/step - loss: 0.6913 - accuracy: 0.5318
Epoch 89/100
804/804 [==============================] - ETA: 0s - loss: 0.6878 - accuracy: 0.56 - ETA: 0s - loss: 0.6930 - accuracy: 0.51 - ETA: 0s - loss: 0.6923 - accuracy: 0.52 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - 

804/804 [==============================] - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6898 - accuracy: 0.54 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6907 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6916 - accuracy: 0.53 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - 1s 982us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 99/100
804/804 [==============================] - ETA: 0s - loss: 0.7007 - accuracy: 0.46 - ETA: 0s - loss: 0.6921 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - los

In [83]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8105 - accuracy: 0.5341
Loss: 0.8105095028877258, Accuracy: 0.5341107845306396


## Attempt 2 - Additional hidden layers added

In [90]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 40
hidden_nodes_layer4 = 20

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 100)               4500      
_________________________________________________________________
dense_64 (Dense)             (None, 80)                8080      
_________________________________________________________________
dense_65 (Dense)             (None, 40)                3240      
_________________________________________________________________
dense_66 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_67 (Dense)             (None, 1)                 21        
Total params: 16,661
Trainable params: 16,661
Non-trainable params: 0
_________________________________________________________________


In [85]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn2.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - ETA: 0s - loss: 603.0552 - accuracy: 0.25 - ETA: 0s - loss: 9023.2480 - accuracy: 0.496 - ETA: 0s - loss: 6099.4448 - accuracy: 0.493 - ETA: 0s - loss: 11655.0137 - accuracy: 0.48 - ETA: 0s - loss: 13145.0479 - accuracy: 0.49 - ETA: 0s - loss: 12240.9043 - accuracy: 0.48 - ETA: 0s - loss: 10977.2100 - accuracy: 0.48 - ETA: 0s - loss: 10527.2656 - accuracy: 0.48 - ETA: 0s - loss: 9400.2930 - accuracy: 0.4902 - ETA: 0s - loss: 11168.1611 - accuracy: 0.49 - ETA: 0s - loss: 12177.2119 - accuracy: 0.48 - ETA: 0s - loss: 11246.2188 - accuracy: 0.48 - ETA: 0s - loss: 10313.6104 - accuracy: 0.48 - ETA: 0s - loss: 9694.8945 - accuracy: 0.4852 - ETA: 0s - loss: 9640.6318 - accuracy: 0.486 - ETA: 0s - loss: 10684.1260 - accuracy: 0.48 - ETA: 0s - loss: 10150.7217 - accuracy: 0.48 - ETA: 0s - loss: 9806.4990 - accuracy: 0.4889 - ETA: 0s - loss: 9345.3604 - accuracy: 0.486 - 1s 1ms/step - loss: 9119.8857 - accuracy: 0.4853
Epoch 2/100
804/804 [

804/804 [==============================] - ETA: 0s - loss: 0.7014 - accuracy: 0.59 - ETA: 0s - loss: 2.2236 - accuracy: 0.50 - ETA: 0s - loss: 1.8117 - accuracy: 0.52 - ETA: 0s - loss: 24.9880 - accuracy: 0.529 - ETA: 0s - loss: 22.2956 - accuracy: 0.528 - ETA: 0s - loss: 17.6590 - accuracy: 0.528 - ETA: 0s - loss: 14.8614 - accuracy: 0.528 - ETA: 0s - loss: 12.9756 - accuracy: 0.526 - ETA: 0s - loss: 11.4697 - accuracy: 0.526 - ETA: 0s - loss: 10.2442 - accuracy: 0.526 - ETA: 0s - loss: 9.2497 - accuracy: 0.526 - ETA: 0s - loss: 8.4894 - accuracy: 0.52 - ETA: 0s - loss: 7.7866 - accuracy: 0.53 - ETA: 0s - loss: 7.2109 - accuracy: 0.53 - ETA: 0s - loss: 6.7223 - accuracy: 0.53 - ETA: 0s - loss: 6.3263 - accuracy: 0.53 - ETA: 0s - loss: 5.9727 - accuracy: 0.53 - ETA: 0s - loss: 5.6929 - accuracy: 0.53 - ETA: 0s - loss: 5.4173 - accuracy: 0.53 - 1s 1ms/step - loss: 5.3887 - accuracy: 0.5328
Epoch 10/100
804/804 [==============================] - ETA: 0s - loss: 0.6857 - accuracy: 0.56 - 

804/804 [==============================] - ETA: 0s - loss: 0.7079 - accuracy: 0.43 - ETA: 0s - loss: 0.6891 - accuracy: 0.54 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6906 - accuracy: 0.53 - ETA: 0s - loss: 0.6907 - accuracy: 0.53 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6910 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 19/100
804/804 [==============================] - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss:

804/804 [==============================] - ETA: 0s - loss: 0.6954 - accuracy: 0.50 - ETA: 0s - loss: 0.6922 - accuracy: 0.52 - ETA: 0s - loss: 0.6931 - accuracy: 0.51 - ETA: 0s - loss: 0.6931 - accuracy: 0.51 - ETA: 0s - loss: 0.6928 - accuracy: 0.51 - ETA: 0s - loss: 0.6925 - accuracy: 0.52 - ETA: 0s - loss: 0.6924 - accuracy: 0.52 - ETA: 0s - loss: 0.6922 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6910 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - 1s 944us/step - loss: 0.6911 - accuracy: 0.5318
Epoch 29/100
804/804 [==============================] - ETA: 0s - loss: 0.6826 - accuracy: 0.59 - ETA: 0s - loss: 0.6919 - accuracy: 0.52 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 0.6939 - accuracy: 0.50 - ETA: 0s - loss: 0.6908 - accuracy: 0.54 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6907 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - 1s 972us/step - loss: 0.6913 - accuracy: 0.5318
Epoch 39/100
804/804 [==============================] - ETA: 0s - loss: 0.6688 - accuracy: 0.68 - ETA: 0s - loss: 0.6897 - accuracy: 0.54 - ETA: 0s - loss: 0.6920 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - los

...

804/804 [==============================] - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6906 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - 1s 988us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 59/100
804/804 [==============================] - ETA: 0s - loss: 0.6875 - accuracy: 0.56 - ETA: 0s - loss: 0.6899 - accuracy: 0.54 - ETA: 0s - loss: 0.6898 - accuracy: 0.54 - ETA: 0s - loss: 0.6904 - accuracy: 0.53 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 0.6807 - accuracy: 0.62 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6906 - accuracy: 0.53 - ETA: 0s - loss: 0.6910 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6910 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 69/100
804/804 [==============================] - ETA: 0s - loss: 0.7006 - accuracy: 0.46 - ETA: 0s - loss:

804/804 [==============================] - ETA: 0s - loss: 0.6750 - accuracy: 0.62 - ETA: 0s - loss: 0.6923 - accuracy: 0.52 - ETA: 0s - loss: 0.6899 - accuracy: 0.54 - ETA: 0s - loss: 0.6910 - accuracy: 0.53 - ETA: 0s - loss: 0.6904 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - 1s 954us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 79/100
804/804 [==============================] - ETA: 0s - loss: 0.6775 - accuracy: 0.62 - ETA: 0s - loss: 0.6918 - accuracy: 0.52 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6906 - accuracy: 0.53 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 0.6996 - accuracy: 0.46 - ETA: 0s - loss: 0.6902 - accuracy: 0.53 - ETA: 0s - loss: 0.6902 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6910 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6909 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - 1s 916us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 89/100
804/804 [==============================] - ETA: 0s - loss: 0.6876 - accuracy: 0.56 - ETA: 0s - loss: 0.6885 - accuracy: 0.55 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6910 - accuracy: 0.53 - ETA: 0s - loss: 0.6908 - accuracy: 0.53 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6929 - accuracy: 0.52 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6903 - accuracy: 0.53 - ETA: 0s - loss: 0.6901 - accuracy: 0.53 - ETA: 0s - loss: 0.6905 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6917 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6916 - accuracy: 0.52 - ETA: 0s - loss: 0.6915 - accuracy: 0.52 - ETA: 0s - loss: 0.6914 - accuracy: 0.53 - ETA: 0s - loss: 0.6912 - accuracy: 0.53 - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - ETA: 0s - loss: 0.6911 - accuracy: 0.53 - 1s 1ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 98/100
804/804 [==============================] - ETA: 0s - loss:

In [86]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6888 - accuracy: 0.5343
Loss: 0.6888319253921509, Accuracy: 0.5343440175056458


## Attempt 3 - The activation function of hidden layers is changed 

In [111]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 25
hidden_nodes_layer4 = 10

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn3.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_98 (Dense)             (None, 100)               4500      
_________________________________________________________________
dense_99 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_100 (Dense)            (None, 25)                1275      
_________________________________________________________________
dense_101 (Dense)            (None, 10)                260       
_________________________________________________________________
dense_102 (Dense)            (None, 1)                 11        
Total params: 11,096
Trainable params: 11,096
Non-trainable params: 0
_________________________________________________________________


In [112]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn3.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - ETA: 0s - loss: 7.2304 - accuracy: 0.53 - ETA: 0s - loss: 8.3917 - accuracy: 0.45 - ETA: 0s - loss: 8.4117 - accuracy: 0.45 - ETA: 0s - loss: 8.2548 - accuracy: 0.46 - ETA: 0s - loss: 8.2601 - accuracy: 0.46 - ETA: 0s - loss: 8.2043 - accuracy: 0.46 - ETA: 0s - loss: 8.1880 - accuracy: 0.46 - ETA: 0s - loss: 8.1680 - accuracy: 0.47 - ETA: 0s - loss: 8.2006 - accuracy: 0.46 - ETA: 0s - loss: 8.1977 - accuracy: 0.46 - ETA: 0s - loss: 8.1761 - accuracy: 0.46 - ETA: 0s - loss: 8.1599 - accuracy: 0.47 - ETA: 0s - loss: 8.1548 - accuracy: 0.47 - ETA: 0s - loss: 8.1770 - accuracy: 0.46 - ETA: 0s - loss: 8.2001 - accuracy: 0.46 - ETA: 0s - loss: 8.2074 - accuracy: 0.46 - ETA: 0s - loss: 8.2066 - accuracy: 0.46 - 1s 1ms/step - loss: 8.2024 - accuracy: 0.4682
Epoch 2/100
804/804 [==============================] - ETA: 0s - loss: 8.6765 - accuracy: 0.43 - ETA: 0s - loss: 8.4875 - accuracy: 0.44 - ETA: 0s - loss: 8.5375 - accuracy: 0.44 - ETA:

804/804 [==============================] - ETA: 0s - loss: 7.7125 - accuracy: 0.50 - ETA: 0s - loss: 8.1332 - accuracy: 0.47 - ETA: 0s - loss: 8.0981 - accuracy: 0.47 - ETA: 0s - loss: 8.2400 - accuracy: 0.46 - ETA: 0s - loss: 8.2869 - accuracy: 0.46 - ETA: 0s - loss: 8.2035 - accuracy: 0.46 - ETA: 0s - loss: 8.2199 - accuracy: 0.46 - ETA: 0s - loss: 8.2035 - accuracy: 0.46 - ETA: 0s - loss: 8.1967 - accuracy: 0.46 - ETA: 0s - loss: 8.1826 - accuracy: 0.46 - ETA: 0s - loss: 8.1687 - accuracy: 0.47 - ETA: 0s - loss: 8.1856 - accuracy: 0.46 - ETA: 0s - loss: 8.2151 - accuracy: 0.46 - ETA: 0s - loss: 8.1877 - accuracy: 0.46 - ETA: 0s - loss: 8.2014 - accuracy: 0.46 - 1s 929us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 13/100
804/804 [==============================] - ETA: 0s - loss: 8.1945 - accuracy: 0.46 - ETA: 0s - loss: 8.1517 - accuracy: 0.47 - ETA: 0s - loss: 8.1394 - accuracy: 0.47 - ETA: 0s - loss: 8.3026 - accuracy: 0.46 - ETA: 0s - loss: 8.2030 - accuracy: 0.46 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 8.1945 - accuracy: 0.46 - ETA: 0s - loss: 8.5269 - accuracy: 0.44 - ETA: 0s - loss: 8.4316 - accuracy: 0.45 - ETA: 0s - loss: 8.3300 - accuracy: 0.46 - ETA: 0s - loss: 8.2952 - accuracy: 0.46 - ETA: 0s - loss: 8.3131 - accuracy: 0.46 - ETA: 0s - loss: 8.2799 - accuracy: 0.46 - ETA: 0s - loss: 8.2783 - accuracy: 0.46 - ETA: 0s - loss: 8.2350 - accuracy: 0.46 - ETA: 0s - loss: 8.2179 - accuracy: 0.46 - ETA: 0s - loss: 8.1694 - accuracy: 0.47 - ETA: 0s - loss: 8.1952 - accuracy: 0.46 - ETA: 0s - loss: 8.1952 - accuracy: 0.46 - ETA: 0s - loss: 8.2034 - accuracy: 0.46 - 1s 876us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 25/100
804/804 [==============================] - ETA: 0s - loss: 7.7125 - accuracy: 0.50 - ETA: 0s - loss: 8.3416 - accuracy: 0.45 - ETA: 0s - loss: 8.2975 - accuracy: 0.46 - ETA: 0s - loss: 8.2937 - accuracy: 0.46 - ETA: 0s - loss: 8.3007 - accuracy: 0.46 - ETA: 0s - loss: 8.2696 - accuracy: 0.46 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 8.6765 - accuracy: 0.43 - ETA: 0s - loss: 8.3298 - accuracy: 0.46 - ETA: 0s - loss: 8.2148 - accuracy: 0.46 - ETA: 0s - loss: 8.1268 - accuracy: 0.47 - ETA: 0s - loss: 8.1419 - accuracy: 0.47 - ETA: 0s - loss: 8.0820 - accuracy: 0.47 - ETA: 0s - loss: 8.0807 - accuracy: 0.47 - ETA: 0s - loss: 8.1105 - accuracy: 0.47 - ETA: 0s - loss: 8.0683 - accuracy: 0.47 - ETA: 0s - loss: 8.1054 - accuracy: 0.47 - ETA: 0s - loss: 8.1142 - accuracy: 0.47 - ETA: 0s - loss: 8.1471 - accuracy: 0.47 - ETA: 0s - loss: 8.1614 - accuracy: 0.47 - ETA: 0s - loss: 8.1633 - accuracy: 0.47 - ETA: 0s - loss: 8.1952 - accuracy: 0.46 - ETA: 0s - loss: 8.2018 - accuracy: 0.46 - 1s 961us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 36/100
804/804 [==============================] - ETA: 0s - loss: 5.7844 - accuracy: 0.62 - ETA: 0s - loss: 8.1672 - accuracy: 0.47 - ETA: 0s - loss: 8.2177 - accuracy: 0.46 - ETA: 0s - loss: 8.2625 - accuracy: 0.46 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 7.7125 - accuracy: 0.50 - ETA: 0s - loss: 7.9309 - accuracy: 0.48 - ETA: 0s - loss: 8.1041 - accuracy: 0.47 - ETA: 0s - loss: 8.1667 - accuracy: 0.47 - ETA: 0s - loss: 8.2212 - accuracy: 0.46 - ETA: 0s - loss: 8.2236 - accuracy: 0.46 - ETA: 0s - loss: 8.1397 - accuracy: 0.47 - ETA: 0s - loss: 8.1771 - accuracy: 0.46 - ETA: 0s - loss: 8.2202 - accuracy: 0.46 - ETA: 0s - loss: 8.1996 - accuracy: 0.46 - ETA: 0s - loss: 8.2082 - accuracy: 0.46 - ETA: 0s - loss: 8.2236 - accuracy: 0.46 - ETA: 0s - loss: 8.2035 - accuracy: 0.46 - ETA: 0s - loss: 8.2011 - accuracy: 0.46 - 1s 837us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 48/100
804/804 [==============================] - ETA: 0s - loss: 10.1226 - accuracy: 0.343 - ETA: 0s - loss: 8.0490 - accuracy: 0.478 - ETA: 0s - loss: 8.1482 - accuracy: 0.47 - ETA: 0s - loss: 8.2412 - accuracy: 0.46 - ETA: 0s - loss: 8.2717 - accuracy: 0.46 - ETA: 0s - loss: 8.2055 - accuracy: 0.46 - ETA: 0s - 

804/804 [==============================] - ETA: 0s - loss: 7.2304 - accuracy: 0.53 - ETA: 0s - loss: 8.2529 - accuracy: 0.46 - ETA: 0s - loss: 8.2203 - accuracy: 0.46 - ETA: 0s - loss: 8.1994 - accuracy: 0.46 - ETA: 0s - loss: 8.2019 - accuracy: 0.46 - ETA: 0s - loss: 8.1752 - accuracy: 0.47 - ETA: 0s - loss: 8.1230 - accuracy: 0.47 - ETA: 0s - loss: 8.1385 - accuracy: 0.47 - ETA: 0s - loss: 8.1362 - accuracy: 0.47 - ETA: 0s - loss: 8.1633 - accuracy: 0.47 - ETA: 0s - loss: 8.1745 - accuracy: 0.47 - ETA: 0s - loss: 8.1736 - accuracy: 0.47 - ETA: 0s - loss: 8.2001 - accuracy: 0.46 - 1s 789us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 60/100
804/804 [==============================] - ETA: 0s - loss: 9.1586 - accuracy: 0.40 - ETA: 0s - loss: 8.1142 - accuracy: 0.47 - ETA: 0s - loss: 8.2129 - accuracy: 0.46 - ETA: 0s - loss: 8.1379 - accuracy: 0.47 - ETA: 0s - loss: 8.2184 - accuracy: 0.46 - ETA: 0s - loss: 8.2373 - accuracy: 0.46 - ETA: 0s - loss: 8.2584 - accuracy: 0.46 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 6.7484 - accuracy: 0.56 - ETA: 0s - loss: 8.0192 - accuracy: 0.48 - ETA: 0s - loss: 8.0768 - accuracy: 0.47 - ETA: 0s - loss: 8.1358 - accuracy: 0.47 - ETA: 0s - loss: 8.1485 - accuracy: 0.47 - ETA: 0s - loss: 8.1901 - accuracy: 0.46 - ETA: 0s - loss: 8.1651 - accuracy: 0.47 - ETA: 0s - loss: 8.1872 - accuracy: 0.46 - ETA: 0s - loss: 8.1687 - accuracy: 0.47 - ETA: 0s - loss: 8.1528 - accuracy: 0.47 - ETA: 0s - loss: 8.1871 - accuracy: 0.46 - ETA: 0s - loss: 8.1717 - accuracy: 0.47 - ETA: 0s - loss: 8.1939 - accuracy: 0.46 - 1s 781us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 72/100
804/804 [==============================] - ETA: 0s - loss: 6.7484 - accuracy: 0.56 - ETA: 0s - loss: 7.9754 - accuracy: 0.48 - ETA: 0s - loss: 8.1463 - accuracy: 0.47 - ETA: 0s - loss: 8.1471 - accuracy: 0.47 - ETA: 0s - loss: 8.1681 - accuracy: 0.47 - ETA: 0s - loss: 8.2035 - accuracy: 0.46 - ETA: 0s - loss: 8.2145 - accuracy: 0.46 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 5.7844 - accuracy: 0.62 - ETA: 0s - loss: 8.2909 - accuracy: 0.46 - ETA: 0s - loss: 8.1945 - accuracy: 0.46 - ETA: 0s - loss: 8.3333 - accuracy: 0.45 - ETA: 0s - loss: 8.2970 - accuracy: 0.46 - ETA: 0s - loss: 8.2677 - accuracy: 0.46 - ETA: 0s - loss: 8.2402 - accuracy: 0.46 - ETA: 0s - loss: 8.2434 - accuracy: 0.46 - ETA: 0s - loss: 8.2188 - accuracy: 0.46 - ETA: 0s - loss: 8.1945 - accuracy: 0.46 - ETA: 0s - loss: 8.1984 - accuracy: 0.46 - ETA: 0s - loss: 8.1623 - accuracy: 0.47 - ETA: 0s - loss: 8.1952 - accuracy: 0.46 - ETA: 0s - loss: 8.1945 - accuracy: 0.46 - 1s 856us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 84/100
804/804 [==============================] - ETA: 0s - loss: 6.7484 - accuracy: 0.56 - ETA: 0s - loss: 8.0455 - accuracy: 0.47 - ETA: 0s - loss: 8.1299 - accuracy: 0.47 - ETA: 0s - loss: 8.1592 - accuracy: 0.47 - ETA: 0s - loss: 8.1901 - accuracy: 0.46 - ETA: 0s - loss: 8.2296 - accuracy: 0.46 - ETA: 0s - los

804/804 [==============================] - ETA: 0s - loss: 9.1586 - accuracy: 0.40 - ETA: 0s - loss: 7.7940 - accuracy: 0.49 - ETA: 0s - loss: 8.1315 - accuracy: 0.47 - ETA: 0s - loss: 8.1744 - accuracy: 0.47 - ETA: 0s - loss: 8.2133 - accuracy: 0.46 - ETA: 0s - loss: 8.1612 - accuracy: 0.47 - ETA: 0s - loss: 8.1616 - accuracy: 0.47 - ETA: 0s - loss: 8.1794 - accuracy: 0.46 - ETA: 0s - loss: 8.1917 - accuracy: 0.46 - ETA: 0s - loss: 8.2165 - accuracy: 0.46 - ETA: 0s - loss: 8.1975 - accuracy: 0.46 - ETA: 0s - loss: 8.2152 - accuracy: 0.46 - ETA: 0s - loss: 8.2231 - accuracy: 0.46 - 1s 805us/step - loss: 8.2024 - accuracy: 0.4682
Epoch 95/100
804/804 [==============================] - ETA: 0s - loss: 7.7125 - accuracy: 0.50 - ETA: 0s - loss: 8.0338 - accuracy: 0.47 - ETA: 0s - loss: 8.0759 - accuracy: 0.47 - ETA: 0s - loss: 8.0504 - accuracy: 0.47 - ETA: 0s - loss: 8.1536 - accuracy: 0.47 - ETA: 0s - loss: 8.1606 - accuracy: 0.47 - ETA: 0s - loss: 8.1217 - accuracy: 0.47 - ETA: 0s - los

In [113]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 3.0485 - accuracy: 0.5718
Loss: 3.048489809036255, Accuracy: 0.571778416633606


In [114]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimzation.h5")